In [1]:
import os
os.environ["SSL_CERT_FILE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"
os.environ["REQUESTS_CA_BUNDLE"] = "/mnt/d/Travel Assistant/Musafir/Fortinet_CA_SSL(15).cer"

In [2]:
from qdrant_client import QdrantClient, models
import requests
from fastembed import TextEmbedding
import json
import random
import pandas as pd

/mnt/d/Travel Assistant/Musafir/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#connecting to local Qdrant instance
qdrant_client = QdrantClient(url="http://localhost:6333")

/tmp/ipykernel_3577/2923977033.py:2: UserWarning: Qdrant client version 1.15.1 is incompatible with server version 1.12.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  qdrant_client = QdrantClient(url="http://localhost:6333")


In [4]:
qdrant_client

In [5]:
df = pd.read_csv("../data/processed_data/all_cities_combined_clean.csv")

In [6]:
documents = df.to_dict(orient='records')

In [7]:
documents[365:370]

[{'id': 365,
  'city': 'Rome',
  'section': 'See',
  'subsection': 'Rome for kids',
  'text': "Children's Museum . Via Flaminia 82. Just north of Piazza del Popolo. Controlled entrance at 10.00, 12,00, 15.00 and 17.00 for visits lasting 1 hour 45 minutes. Closed Mondays and for much of August. Best to check the web site for up-to-date info and to book in advance. Hands-on science, mainly for pre-teens, housed in a former tram-car depot."},
 {'id': 366,
  'city': 'Rome',
  'section': 'Eat',
  'subsection': 'Pricing',
  'text': 'Chinese restaurants are still quite cheap but other ethnic restaurants (Thai, Indian) are generally expensive (think €30 upwards per person). Sushi is very expensive (€40 minimum per person).'},
 {'id': 367,
  'city': 'Rome',
  'section': 'Eat',
  'subsection': nan,
  'text': 'Coda alla vaccinara - Oxtail stew'},
 {'id': 368,
  'city': 'Rome',
  'section': 'Sleep',
  'subsection': nan,
  'text': 'Colosseo'},
 {'id': 369,
  'city': 'Rome',
  'section': 'Districts'

In [8]:
emb_models = TextEmbedding.list_supported_models()

In [9]:
emb_models[0]

{'model': 'BAAI/bge-base-en',
 'sources': {'hf': 'Qdrant/fast-bge-base-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.42,
 'additional_files': [],
 'dim': 768,
 'tasks': {}}

In [10]:
len(emb_models)

30

In [11]:
# check models with 512 dim 
EMBEDDING_DIMENSIONALITY = 512
for model in emb_models:
    if model['dim'] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent = 2))



# for model in TextEmbedding.list_supported_models():
#     if model["dim"] == EMBEDDING_DIMENSIONALITY:
#         print(json.dumps(model, indent = 2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [12]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

# Create collection 

In [14]:
# Define the collection name
collection_name = "traveller-rag"
# Delete a connection 
if qdrant_client.collection_exists(collection_name=collection_name):
    qdrant_client.delete_collection(collection_name=collection_name)
    print(f"Delete the currrent collection {collection_name}")


Delete the currrent collection traveller-rag


In [15]:
# Create the collection with specified vector parameters
qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size = EMBEDDING_DIMENSIONALITY, 
        distance=models.Distance.COSINE
    )
)

ResponseHandlingException: timed out

In [16]:
df.columns

Index(['id', 'city', 'section', 'subsection', 'text'], dtype='object')

# Create, Embed & Insert Points into the Collection


In [17]:
points = []
id = 0

for doc in documents:
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "city": doc['city'],
            "section": doc['section'],
            "subsection": doc['subsection']
            
        }
    )
    points.append(point)

    id += 1



In [18]:
qdrant_client.upsert(
    collection_name=collection_name, 
    points=points
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Running a Similarity Search


In [22]:
def qdrant_search(query, limit=1):

    results = qdrant_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results 


In [23]:
# Generate random part of the doc data 
city_piece= random.choice(documents)
print(json.dumps(city_piece, indent=2))

{
  "id": 41,
  "city": "Cairo",
  "section": "Districts",
  "subsection": NaN,
  "text": "Greater Cairo is vast; with more than 20 million people (2018), it's the largest metropolitan area in Africa and the Middle East."
}


In [24]:
result =  qdrant_search(city_piece['text'])

In [25]:
result

QueryResponse(points=[ScoredPoint(id=41, version=0, score=1.0, payload={'text': "Greater Cairo is vast; with more than 20 million people (2018), it's the largest metropolitan area in Africa and the Middle East.", 'city': 'Cairo', 'section': 'Districts', 'subsection': None}, vector=None, shard_key=None, order_value=None)])

In [26]:
print(f"Question:\n{city_piece['section']}\n")
print("Top Retrieved Answer:\n{}\n".format(result.points[0].payload['text']))
print("Original Answer:\n{}".format(city_piece['text']))

Question:
Districts

Top Retrieved Answer:
Greater Cairo is vast; with more than 20 million people (2018), it's the largest metropolitan area in Africa and the Middle East.

Original Answer:
Greater Cairo is vast; with more than 20 million people (2018), it's the largest metropolitan area in Africa and the Middle East.


In [27]:
print(qdrant_search("Where I can eat halal food in London?").points[0].payload['text'])

Due to the mix of cultures and religions, many London restaurants cater well for religious dietary requirements. The most common signs are for Halal and Kosher meat, from burger joints to nice restaurants. There are lots of Halal restaurants and shops all over London including Whitechapel Rd and Brick Lane in the East End , Bayswater, Edgware Rd and Paddington and in many parts of north London . There are plenty of Kosher restaurants in Golders Green , Edgware and Stamford Hill along with some central delis such as on Charing Cross Road.


In [28]:
query = "Where I can eat halal food in London?"
qdrant_search(query)

QueryResponse(points=[ScoredPoint(id=135, version=0, score=0.9013014, payload={'text': 'Due to the mix of cultures and religions, many London restaurants cater well for religious dietary requirements. The most common signs are for Halal and Kosher meat, from burger joints to nice restaurants. There are lots of Halal restaurants and shops all over London including Whitechapel Rd and Brick Lane in the East End , Bayswater, Edgware Rd and Paddington and in many parts of north London . There are plenty of Kosher restaurants in Golders Green , Edgware and Stamford Hill along with some central delis such as on Charing Cross Road.', 'city': 'London', 'section': 'Eat', 'subsection': 'Religious'}, vector=None, shard_key=None, order_value=None)])

In [29]:
from mistralai import Mistral
from mistralai.models import UserMessage
import os
from dotenv import load_dotenv


In [30]:
# loads variables from .env
load_dotenv()  

True

In [31]:
api_key = os.getenv("API_KEY")

In [32]:
llm_client = Mistral(api_key = api_key)

In [47]:
def build_prompt(query, search_results):
    context_template = "Q: {question}\n A: {text}"

    context_parts = []
    for point in search_results.points:  # iterate over .points
        payload = point.payload
        context_parts.append(
            context_template.format(
                question=query,
                text=payload.get("text", "")
            )
        )

    context = "\n\n".join(context_parts)
    
    prompt_template = """
You're a travel assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}

    """.strip()
    
    prompt = prompt_template.format(question=query, context=context)
    return prompt


In [51]:
def llm(prompt):
    response = llm_client.chat.complete(
        model= "ministral-8b-latest",
        messages=[UserMessage(content=prompt)],
    )

    return response.choices[0].message.content

In [52]:
def rag(query):
    search_results = qdrant_search(query=query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [53]:
answer = rag(query)
print(answer)

Based on the context, you can eat halal food in London at various locations, including:

- Whitechapel Road and Brick Lane in the East End
- Bayswater, Edgware Road, and Paddington
- Many parts of north London


# Running a Similarity Search with Filters


In [55]:
qdrant_client.create_payload_index(
    collection_name=collection_name, 
    field_name="city",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [56]:
# Update search function 

def search_in_city(query, city, limit):
    results=qdrant_client.query_points(
        collection_name=collection_name, 
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter(#filter by city name
            must=[
                models.FieldCondition(
                    key="city",
                    match=models.MatchValue(value=city)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True # get metadata in teh results
    )
    return results

In [57]:
print(search_in_city("Designa one day trip?", "Seoul", 1).points[0].payload['text'])

There are many budget accommodation places across Seoul. Hongdae, Itaewon, Myeongdong and Jongno (Hanok area) are traditional hot spots for Foreign Individual Travelers (FIT). Furthermore, Gangnam is emerging thanks to the huge success of the eponymous song.
Hongdae, Sinchon area is in university area. Yonsei Univ., Ehwa woman's Univ., Hongik Univ. and Sogang Univ. are around this area. so there are many restaurants, bar, club and shopping center and easy to be reached from Incheon Airport by limousine bus and Arex (Airport express train) in 1 hour.


In [67]:
def rag_city(query, city, limit):
    search_results = search_in_city(query=query, city = city, limit = limit)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)

    return answer

In [71]:
answer = rag_city("Designa one day trip?", 'Seoul', 1)
print(answer)

Based on the provided context, here's a suggested one-day trip itinerary for Seoul:

1. **Morning:**
   - Start your day in **Hongdae**, a vibrant neighborhood known for its street art, cafes, and shops. Explore the area and have breakfast at one of the local cafes.
   - Visit **Yonsei University** and **Ehwa Women's University** nearby, if you're interested in seeing the university life.

2. **Afternoon:**
   - Head to **Myeongdong**, a popular shopping district. You can find various street food, shops, and restaurants here.
   - Visit **Jongno (Hanok area)** to explore traditional Korean houses and experience the local culture.

3. **Evening:**
   - For dinner, consider **Itaewon**, known for its international cuisine and lively atmosphere.
   - End your day in **Gangnam**, the trendy district made famous by the song "Gangnam Style". Enjoy the nightlife and shopping.

This itinerary covers various aspects of Seoul, including culture, shopping, and nightlife, and is designed to be com

# hybrid_search

## Sparse vector search with BM25


In [78]:
import uuid

In [73]:
# Define the collection name
collection_name = "traveller-sparse"

# Create the collection with specified sparse vector parameters
qdrant_client.create_collection(
    collection_name = collection_name,
    sparse_vectors_config={
        "bm25":models.SparseVectorParams(
            modifier=models.Modifier.IDF
        )
    }
)

ResponseHandlingException: timed out

In [113]:
# Send the points to the collection
qdrant_client.upsert(
    collection_name = collection_name,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25": models.Document(
                    text=doc["text"],
                    model="Qdrant/bm25",
                )
            }, 
            payload={
            "id": doc['id'],
            "text": doc['text'],
            "city": doc['city'],
            "section": doc['section'],
            "subsection": doc['subsection']

            }
        )
        for doc in documents
    ]
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

## Running sparse vector search with BM25

In [114]:
def bm_search(query: str, limit: int =1) -> list[models.ScoredPoint]:
    results = qdrant_client.query_points(
        collection_name = collection_name, 
        query=models.Document(
            text=query, 
            model="Qdrant/bm25",
        ),
        using="bm25",
        limit=limit,
        with_payload=True
    )
    return results.points

In [115]:
results = bm_search("Qdrant")
results

[]

In [116]:
results = bm_search("Roma")
print(results[0].payload['text'])

Happy Valley ( In the hills north of the city at Via Prato della Corte 1915, Prima Porta-Cassia Bis, Roma. Take exit no 5 from Rome's ring road and head towards Cassia-Veientana. If you take public transport, take the ground-level Roma-Nord Subway leaving from Piazza Flaminia towards Prima Porta and wait for the free shuttle bus service. ), ☏ +39 06-33626401 , fax : +39 06-33613800 . It has a pool, a bar, a restaurant and a minimarket.


In [117]:
results[0].score

6.70714

In [118]:
random.seed(202506)
city_piece=random.choice(documents)
print(json.dumps(city_piece, indent=2))

{
  "id": 517,
  "city": "Seoul",
  "section": "Buy",
  "subsection": "Fashion",
  "text": "Seongsu also is rapidly becoming the center of fashion in Seoul. It was once an industrial complex, but now it's home to new fashions such as hip-hop, reggae, vintage, and gukdu, as the abandoned factories have been redecorated by young people. Nearby is Ttukseom Hangang Park, where you can enjoy Korea's famous fried chicken and beer."
}


In [119]:
results = bm_search(city_piece["text"])
print(results[0].payload["text"])

Seongsu also is rapidly becoming the center of fashion in Seoul. It was once an industrial complex, but now it's home to new fashions such as hip-hop, reggae, vintage, and gukdu, as the abandoned factories have been redecorated by young people. Nearby is Ttukseom Hangang Park, where you can enjoy Korea's famous fried chicken and beer.


##  Qdrant Universal Query API - prefetching

In [128]:
# Create the collection with both vector types
collection_name_mx="traveller-sparse-and-dense"
qdrant_client.create_collection(
    collection_name=collection_name_mx,
    vectors_config={
        # Named dense vector for jinaai/jina-embeddings-v2-small-en
        'jina-small':models.VectorParams(
            size=512,
            distance=models.Distance.COSINE
        ),
    },
    sparse_vectors_config={
        "bm25":models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

ResponseHandlingException: timed out

In [129]:
qdrant_client.upsert(
    collection_name=collection_name_mx,
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=doc["text"],
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                "bm25": models.Document(
                    text=doc["text"], 
                    model="Qdrant/bm25",
                ),
            },
            payload={
            "id": doc['id'],
            "text": doc['text'],
            "city": doc['city'],
            "section": doc['section'],
            "subsection": doc['subsection']}

        )
        for doc in documents
    ]
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [130]:
def multi_stage_search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = qdrant_client.query_points(
        collection_name="traveller-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                # Prefetch ten times more results, then
                # expected to return, so we can really rerank
                limit=(10 * limit),
            ),
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25", 
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [131]:
print(json.dumps(city_piece, indent=2))

{
  "id": 517,
  "city": "Seoul",
  "section": "Buy",
  "subsection": "Fashion",
  "text": "Seongsu also is rapidly becoming the center of fashion in Seoul. It was once an industrial complex, but now it's home to new fashions such as hip-hop, reggae, vintage, and gukdu, as the abandoned factories have been redecorated by young people. Nearby is Ttukseom Hangang Park, where you can enjoy Korea's famous fried chicken and beer."
}


## Building Hybrid Search

In [132]:
def rrf_search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = qdrant_client.query_points(
        collection_name="traveller-sparse-and-dense",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                limit=(5 * limit),
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="Qdrant/bm25",
                ),
                using="bm25",
                limit=(5 * limit),
            ),
        ],
        # Fusion query enables fusion on the prefetched results
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
    )

    return results.points

In [134]:
results = rrf_search(city_piece["text"])
print(json.dumps(city_piece, indent=2))
print(results[0].payload["text"])

{
  "id": 517,
  "city": "Seoul",
  "section": "Buy",
  "subsection": "Fashion",
  "text": "Seongsu also is rapidly becoming the center of fashion in Seoul. It was once an industrial complex, but now it's home to new fashions such as hip-hop, reggae, vintage, and gukdu, as the abandoned factories have been redecorated by young people. Nearby is Ttukseom Hangang Park, where you can enjoy Korea's famous fried chicken and beer."
}
Seongsu also is rapidly becoming the center of fashion in Seoul. It was once an industrial complex, but now it's home to new fashions such as hip-hop, reggae, vintage, and gukdu, as the abandoned factories have been redecorated by young people. Nearby is Ttukseom Hangang Park, where you can enjoy Korea's famous fried chicken and beer.
